In [15]:
import pandas as pd

# df_crims = pd.read_csv('CMX_2024.csv')

# Segun hora del dia
daytime = 12

if daytime >= 0 and daytime < 5:
    df_crims = pd.read_csv('CMX_madrugada.csv')
elif daytime >= 5 and daytime < 10:
    df_crims = pd.read_csv('CMX_mañana.csv')
elif daytime >= 10 and daytime < 13:
    df_crims = pd.read_csv('CMX_mediodia.csv')
elif daytime >= 13 and daytime < 17:
    df_crims = pd.read_csv('CMX_tarde.csv')
elif daytime >= 17 and daytime < 21:
    df_crims = pd.read_csv('CMX_noche.csv')
elif daytime >= 21 and daytime < 24:
    df_crims = pd.read_csv('CMX_medianoche.csv')

display(df_crims.head())

,mes_hecho,fecha_hecho,delito,categoria_delito,latitud,longitud,delito_encoded,categoria_delito_encoded,time_seconds,time_zones,cluster_label
0,3,2083,"ALLANAMIENTO DE MORADA, DESPACHO, OFICINA O ES...",DELITO DE BAJO IMPACTO,19.429890,-99.053063,6,0,9600.0,3,-1
1,3,2083,ROBO A CASA HABITACION SIN VIOLENCIA,DELITO DE BAJO IMPACTO,19.334924,-99.134299,118,0,5400.0,3,-1
2,3,2083,ROBO DE ACCESORIOS DE AUTO,DELITO DE BAJO IMPACTO,19.406539,-99.127682,174,0,9000.0,3,-1
3,3,2083,ROBO DE VEHICULO DE SERVICIO PARTICULAR SIN VI...,ROBO DE VEHÃCULO CON Y SIN VIOLENCIA,19.279583,-99.217773,197,12,7200.0,3,-1
4,3,2083,ROBO A TRANSEUNTE EN VIA PUBLICA CON VIOLENCIA,ROBO A TRANSEUNTE EN VÃA PÃBLICA CON Y SIN V...,19.288396,-99.053909,168,10,12600.0,3,-1


In [16]:
from datetime import datetime
import nltk

# Convert 'fecha_hecho' to datetime
df_crims['fecha_hecho'] = pd.to_datetime(df_crims['fecha_hecho'])

df_last_year = df_crims[df_crims['fecha_hecho'].dt.year == 2024]
df_delict = df_last_year[df_last_year['categoria_delito'] != 'HECHO NO DELICTIVO']

df_bajo_impacto = df_delict[df_delict['categoria_delito'] == 'DELITO DE BAJO IMPACTO']
small_delicts = df_bajo_impacto['delito'].value_counts()
filtered_delicts = small_delicts[small_delicts.index.str.contains('ROBO|ACOSO|HOMICICIO|PANDILLA', case=False)]

display(df_delict.shape)
# Filter out the delicts that are NOT in the filtered_delicts list
df_delict = df_delict[~((df_delict['categoria_delito'] == 'DELITO DE BAJO IMPACTO') & (~df_delict['delito'].isin(filtered_delicts.index)))]
display(df_delict.shape)

df_delict = df_delict.dropna(subset=['latitud', 'longitud'])
display(df_delict.shape)


#df_crimes = df_delict[['delito', 'longitud', 'latitud']]

display(df_delict['categoria_delito'].value_counts())

(0, 11)

(0, 11)

(0, 11)

Series([], Name: count, dtype: int64)

In [17]:
crime_category_to_weight = {
    'DELITO DE BAJO IMPACTO': 1,
    'ROBO A TRANSEUNTE EN VÍA PÚBLICA CON Y SIN VIOLENCIA': 3,
    'ROBO DE VEHÍCULO CON Y SIN VIOLENCIA': 2,
    'ROBO A NEGOCIO CON VIOLENCIA': 3,
    'VIOLACIÓN': 4,
    'ROBO A PASAJERO A BORDO DEL METRO CON Y SIN VIOLENCIA': 2,
    'HOMICIDIO DOLOSO': 5,
    'ROBO A REPARTIDOR CON Y SIN VIOLENCIA': 3,
    'ROBO A PASAJERO A BORDO DE MICROBÚS CON Y SIN VIOLENCIA': 2,
    'LESIONES DOLOSAS POR DISPARO DE ARMA DE FUEGO': 4,
    'ROBO A CASA HABITACIÓN CON VIOLENCIA': 3,
    'ROBO A CUENTAHABIENTE SALIENDO DEL CAJERO CON VIOLENCIA': 3,
    'ROBO A PASAJERO A BORDO DE TAXI CON VIOLENCIA': 3,
    'ROBO A TRANSPORTISTA CON Y SIN VIOLENCIA': 2,
    'SECUESTRO': 5
}

df_crimes_with_weight = df_delict.copy()
df_crimes_with_weight['weight'] = df_crimes_with_weight['categoria_delito'].map(crime_category_to_weight)
df_crimes_with_weight['weight'] = df_crimes_with_weight['weight'].fillna(1)

display(df_crimes_with_weight['weight'].value_counts())

Series([], Name: count, dtype: int64)

In [18]:
df_ponderado = df_crimes_with_weight[['delito', 'categoria_delito', 'longitud', 'latitud', 'weight']]
df_ponderado.loc[df_ponderado['delito'].str.contains('CON', case=True), 'weight'] += 0.5

df_weights = df_ponderado[['latitud','longitud','weight']]
df_weights
display(df_weights['weight'].value_counts())

Series([], Name: count, dtype: int64)

In [19]:
import geojson
from geojson import Feature, FeatureCollection, Point

def df_to_geojson(df):
    features = []
    for _, row in df.iterrows():
        if not pd.isna(row['latitud']) and not pd.isna(row['longitud']):
            point = Point((row['longitud'], row['latitud']))
            features.append(
                Feature(geometry=point, properties={"weight": row['weight']})
            )
    return FeatureCollection(features)

geojson_data = df_to_geojson(df_weights)

# Save to a GeoJSON file
with open('crimes.geojson', 'w') as f:
    geojson.dump(geojson_data, f)

In [20]:
def filter_geojson_by_coordinates(geojson_data, lat_min, lat_max, lon_min, lon_max):
    filtered_features = []
    for feature in geojson_data['features']:
        lon, lat = feature['geometry']['coordinates']
        if lat_min <= lat <= lat_max and lon_min <= lon <= lon_max:
            filtered_features.append(feature)
    
    return geojson.FeatureCollection(filtered_features)

# Example usage
lat_min, lat_max = 19.0, 20.0
lon_min, lon_max = -100.0, -98.0
filtered_geojson_data = filter_geojson_by_coordinates(geojson_data, lat_min, lat_max, lon_min, lon_max)

# Save the filtered GeoJSON to a new file
with open('filtered_crimes.geojson', 'w') as f:
    geojson.dump(filtered_geojson_data, f)

In [21]:
import folium

# Crear un mapa centrado en el área filtrada
center_lat = (lat_min + lat_max) / 2
center_lon = (lon_min + lon_max) / 2
m = folium.Map(location=[center_lat, center_lon], zoom_start=10)

# Añadir el archivo GeoJSON al mapa
folium.GeoJson('filtered_crimes.geojson').add_to(m)

# Mostrar el mapa


In [22]:
import folium

def visualize_map(df):
    # Create a map centered around the average latitude and longitude
    m = folium.Map(location=[df['latitud'].mean(), df['longitud'].mean()], zoom_start=12)
    
    # Add points to the map
    for _, row in df.iterrows():
        if not pd.isna(row['latitud']) and not pd.isna(row['longitud']):
            folium.Marker(
                location=[row['latitud'], row['longitud']],
                popup=row['delito'],
                tooltip=row['categoria_delito']
            ).add_to(m)
    
    return m

# Example usage
map_visualization = visualize_map(df_crims)
map_visualization